In [1]:
import numpy as np
import pandas as pd
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn import preprocessing
from sklearn import utils
import time as t

In [2]:
source_df = pd.read_csv('C:\\Users\\Jonathan McAwesome\\Dropbox\\Projects\\UNICEF_Flood_Prediction_in_Malawi\\Source_Data\\Zindi\\train.csv',index_col='Square_ID')

In [3]:
# Change Date Ranges to Week1,Week2 etc.

column_names = list(source_df.columns)
source_df_2015 =source_df[['X','Y','elevation','LC_Type1_mode','target_2015']]
source_df_2019 =source_df[['X','Y','elevation','LC_Type1_mode']]

x=1

for i in column_names:
    if i[0:6]=='precip':
        #print(type(i[7:11]))
        if i[7:11] == '2014'or i[7:11] =='2015':
            source_df_2015['Week'+str(x)] = source_df[str(i)]
            #source_df.drop(i,axis=1,inplace=True)
            x= x+1
x=1
for i in column_names:
    if i[0:6]=='precip':
        #print(type(i[7:11]))
        if i[7:11] == '2019':
            source_df_2019['Week'+str(x)] = source_df[str(i)]
            #source_df.drop(i,axis=1,inplace=True)
            x= x+1



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
# Rework Catagorical Data to Numerical Location Type.
source_df_2015 = pd.get_dummies(source_df_2015,prefix="lc_type",columns=['LC_Type1_mode'])
source_df_2019 = pd.get_dummies(source_df_2019,prefix="lc_type",columns=['LC_Type1_mode'])

In [5]:
y = list(source_df_2015['target_2015'])
X = source_df_2015.drop('target_2015',axis=1)
dtrain = xgb.DMatrix(X, label=y)


In [6]:
# Max Depth = 3 best so far. Tried 6, 2
param = {'max_depth':3, 'eta':1, 'objective':'binary:logistic' }
# 4 Boosting Rounds Worked Best, tried: 2,3,5,120,60
num_round = 4
xgb_model = xgb.train(param, dtrain, num_round)

In [7]:
dpredict = xgb.DMatrix(source_df_2019)
source_df_2019['target_2019'] = xgb_model.predict(dpredict)

In [8]:
t.localtime()

time.struct_time(tm_year=2020, tm_mon=4, tm_mday=1, tm_hour=18, tm_min=55, tm_sec=1, tm_wday=2, tm_yday=92, tm_isdst=0)

In [9]:
Hour = t.localtime().tm_hour
Minute = t.localtime().tm_min
Year = t.localtime().tm_year
Month = '0'+str(t.localtime().tm_mon)
Day = '0'+str(t.localtime().tm_mday)
source_df_2019['target_2019'].to_csv(f'{Year}{Month}{Day} {Hour}H{Minute} XGBoost_prediction.csv',header=True)